In [ ]:
import pandas as pd
import logging
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras import regularizers
from keras import metrics
from keras.layers import Dense, LSTM, Conv1D, Dropout, MaxPooling1D, Flatten, Flatten, GRU, Average
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.layers.embeddings import Embedding
from keras.models import Sequential
from keras.models import load_model
from keras.optimizers import Adam
from keras.preprocessing import sequence
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import emoji
import re
import glob
logging.basicConfig(format='%(levelname)s %(asctime)s: %(message)s', level=logging.INFO)

In [ ]:
data = pd.read_csv("/kaggle/input/newdataset/new-data-set.csv")
stopWordsFileContent = open("/kaggle/input/stopwords/stopWords.txt", "r", encoding="utf16").readlines()
stemDictFileContent = open("/kaggle/input/stem-dict/stem_dictionary.txt", "r").readlines()
# print("Before", data.shape)
# data = data[pd.notnull(data['full_text_without_emoji'])]
# data = data[pd.notnull(data['label'])]
# print("After:", data.shape)
data.head()

In [ ]:
stemDict = {}
for line in stemDictFileContent:
    words = line.split("\t")
    stemDict[words[0].strip()] = words[1].strip()

In [ ]:
stopWords = []
for line in stopWordsFileContent:
    stopWords.append(line.split("\t")[0])

In [ ]:
data.shape

In [ ]:
def tokenize(text: str) -> list:
    # text characters to split is from: https://github.com/madurangasiriwardena/corpus.sinhala.tools
    emojis = ''.join(emj for emj in emoji.UNICODE_EMOJI.keys())
    tokens = [token for token in
            re.split(r'[.…,‌ ¸‚\"/|—¦”‘\'“’´!@#$%^&*+\-£?˜()\[\]{\}:;–Ê  �‪‬‏0123456789' + emojis + ']', text)
            if token != "" and token not in stopWords]
    stemmedTokens = []
    for t in tokens:
        if(t in stemDict.keys()):
            stemmedTokens.append(stemDict[t])
        else:
            stemmedTokens.append(t)
    return stemmedTokens
tokenize("අනේ හුකන්න කියපන් උන්ට")

In [ ]:
def tokenize_corpus(corpus: list) -> list:
    return [tokenize(text) for text in corpus]
tokenized_corpus = tokenize_corpus(["අනේ හුකන්න කියපන් උන්ට", "පාහර බැල්ලි එනෝ මෙතන මට වැඩ කියාදෙන්න හෙළුවෙන්"])
tokenized_corpus

In [ ]:
MAX_WORD_COUNT = 60 
DATA_SET_CLASSES = {
    0: [0, 1],
    1: [1, 0]
}

In [ ]:
def transform_class_to_one_hot_representation(classes: list):
    return np.array([DATA_SET_CLASSES[cls] for cls in classes])
transform_class_to_one_hot_representation([1,0,1])

In [ ]:
def build_dictionary(corpus_token: list) -> dict:
    word_frequency = {}
    dictionary = {}

    for tweet in corpus_token:
        for token in tweet:
            if token in word_frequency:
                word_frequency[token] += 1
            else:
                word_frequency[token] = 1

    frequencies = list(word_frequency.values())
    unique_words = list(word_frequency.keys())

    # sort words by its frequency
    frequency_indexes = np.argsort(frequencies)[::-1]  # reverse for descending
    for index, frequency_index in enumerate(frequency_indexes):
        # 0 is not used and 1 is for UNKNOWN
        dictionary[unique_words[frequency_index]] = index + 2

    return dictionary
print(tokenized_corpus)
dictionary = build_dictionary(tokenized_corpus)
dictionary

In [ ]:
def transform_to_dictionary_values(corpus_token: list, dictionary: dict) -> list:
    x_corpus = []
    for tweet in corpus_token:
        # 1 is for unknown (not in dictionary)
        x_corpus.append([dictionary[token] if token in dictionary else 1 for token in tweet])

    return x_corpus
transform_to_dictionary_values(tokenized_corpus, dictionary)

In [ ]:
data_set = data.values

In [ ]:
DATA_SET_TEXT = 1
logging.info("Tokenizing the corpus")
corpus_token = tokenize_corpus(data_set[:, DATA_SET_TEXT])

In [ ]:
logging.info("Building the dictionary")
dictionary = build_dictionary(corpus_token)
dictionary_length = len(dictionary) + 2  # 0 is not used and 1 is for UNKNOWN
dictionary_length

In [ ]:
logging.info("Transforming the corpus to dictionary values")
x_corpus = transform_to_dictionary_values(corpus_token, dictionary)

In [ ]:
DATA_SET_CLASS = 2
y_corpus = transform_class_to_one_hot_representation(data_set[:, DATA_SET_CLASS])
y_corpus

In [ ]:
x_corpus = sequence.pad_sequences(x_corpus, maxlen=MAX_WORD_COUNT )
print(len(x_corpus))

In [ ]:
# ################## Deep Neural Network ###################### #
FOLDS_COUNT = 5
MAX_EPOCHS = 5
VALIDATION_TEST_SIZE = 0.12
max_word_count = MAX_WORD_COUNT

# splitting data for 5-fold cross validation
k_fold = StratifiedKFold(n_splits=FOLDS_COUNT, shuffle=True, random_state=18)
# to split, raw format (integer) is required
y_corpus_raw = [0 if cls[1] == 1 else 1 for cls in y_corpus]

# LSTM Model

In [ ]:
fold = 0
cv_scores = []
for train_n_validation_indexes, test_indexes in k_fold.split(x_corpus, y_corpus_raw):
    x_train_n_validation = x_corpus[train_n_validation_indexes]
    y_train_n_validation = y_corpus[train_n_validation_indexes]
    x_test = x_corpus[test_indexes]
    y_test = y_corpus[test_indexes]

    # train and validation data sets
#     x_train, x_valid, y_train, y_valid = train_test_split(x_train_n_validation, y_train_n_validation,
#                                                           test_size=VALIDATION_TEST_SIZE, random_state=94)

    n_timesteps = len(x_train_n_validation)
    n_features = len(x_train_n_validation[0])
    # ################## Deep Neural Network Model ###################### #
    model = Sequential()
    model.add(Embedding(input_dim=dictionary_length, output_dim=60, input_length=max_word_count))
    model.add(LSTM(600))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(2, activation='softmax'))
#     model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    adam_optimizer = Adam(lr=0.001, decay=0.0001)
    model.compile(loss='categorical_crossentropy', optimizer=adam_optimizer, metrics=[metrics.CategoricalAccuracy(), metrics.AUC(), metrics.Precision(), metrics.Recall()])
    
#     print(model.summary())
    # ################## Deep Neural Network Model ###################### #

    best_accuracy = 0
    best_loss = 100000
    best_epoch = 0

    epoch_history = {
        'accuracy': [],
        'val_accuracy': [],
        'loss': [],
        'val_loss': [],
        'auc': [],
        'val_auc': []
    }
    history = model.fit(x=x_train_n_validation, y=y_train_n_validation, epochs=MAX_EPOCHS, batch_size=32,
                            verbose=1, shuffle=True)
    scores = model.evaluate(x_test, y_test, verbose=0)
    print(model.metrics_names)
    print("scores:", scores)
    cv_scores.append(scores)
    fold += 1

In [ ]:
mean_accuracy = 0
mean_auc = 0
mean_precision = 0
mean_recall = 0
for score in cv_scores:
    mean_accuracy += score[1]
    mean_auc += score[2]
    mean_precision += score[3]
    mean_recall += score[4]
mean_accuracy = mean_accuracy / FOLDS_COUNT
mean_auc = mean_auc / FOLDS_COUNT
mean_precision = mean_precision / FOLDS_COUNT
mean_recall = mean_recall / FOLDS_COUNT
mean_f_score = 2*mean_precision*mean_recall/(mean_precision + mean_recall)
print("mean_accuracy", mean_accuracy)
print("mean_auc", mean_auc)
print("mean_precision", mean_precision)
print("mean_recall", mean_recall)
print("mean_f_score", mean_f_score)

# CNN Model

In [ ]:
fold = 0
cv_scores = []
for train_n_validation_indexes, test_indexes in k_fold.split(x_corpus, y_corpus_raw):
    x_train_n_validation = x_corpus[train_n_validation_indexes]
    y_train_n_validation = y_corpus[train_n_validation_indexes]
    x_test = x_corpus[test_indexes]
    y_test = y_corpus[test_indexes]

    # train and validation data sets
#     x_train, x_valid, y_train, y_valid = train_test_split(x_train_n_validation, y_train_n_validation,
#                                                           test_size=VALIDATION_TEST_SIZE, random_state=94)

    n_timesteps = len(x_train_n_validation)
    n_features = len(x_train_n_validation[0])
    # ################## Deep Neural Network Model ###################### #
    model = Sequential()
    model.add(Embedding(input_dim=dictionary_length, output_dim=60, input_length=max_word_count))
    model.add(Conv1D(32, 3, padding='same', activation='relu'))
    model.add(MaxPooling1D())
    model.add(Flatten())
    model.add(Dense(250, activation='relu'))
    model.add(Dense(2, activation='sigmoid'))
#     model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    adam_optimizer = Adam(lr=0.001, decay=0.0001)
    model.compile(loss='binary_crossentropy', optimizer=adam_optimizer, metrics=[metrics.CategoricalAccuracy(), metrics.AUC(), metrics.Precision(), metrics.Recall()])
    
    print(model.summary())
    # ################## Deep Neural Network Model ###################### #

    best_accuracy = 0
    best_loss = 100000
    best_epoch = 0

    epoch_history = {
        'accuracy': [],
        'val_accuracy': [],
        'loss': [],
        'val_loss': [],
        'auc': [],
        'val_auc': []
    }

    history = model.fit(x=x_train_n_validation, y=y_train_n_validation, epochs=MAX_EPOCHS, batch_size=32,
                            verbose=1, shuffle=True)
    scores = model.evaluate(x_test, y_test, verbose=0)
    print(model.metrics_names)
    print("scores:", scores)
    cv_scores.append(scores)

In [ ]:
mean_accuracy = 0
mean_auc = 0
mean_precision = 0
mean_recall = 0
for score in cv_scores:
    mean_accuracy += score[1]
    mean_auc += score[2]
    mean_precision += score[3]
    mean_recall += score[4]
mean_accuracy = mean_accuracy / FOLDS_COUNT
mean_auc = mean_auc / FOLDS_COUNT
mean_precision = mean_precision / FOLDS_COUNT
mean_recall = mean_recall / FOLDS_COUNT
mean_f_score = 2*mean_precision*mean_recall/(mean_precision + mean_recall)
print("mean_accuracy", mean_accuracy)
print("mean_auc", mean_auc)
print("mean_precision", mean_precision)
print("mean_recall", mean_recall)
print("mean_f_score", mean_f_score)

# BiGRU Model

In [ ]:
fold = 0
cv_scores = []
for train_n_validation_indexes, test_indexes in k_fold.split(x_corpus, y_corpus_raw):
    x_train_n_validation = x_corpus[train_n_validation_indexes]
    y_train_n_validation = y_corpus[train_n_validation_indexes]
    x_test = x_corpus[test_indexes]
    y_test = y_corpus[test_indexes]


    n_timesteps = len(x_train_n_validation)
    n_features = len(x_train_n_validation[0])
    # ################## Deep Neural Network Model ###################### #
    model = Sequential()
    model.add(Embedding(input_dim=dictionary_length, output_dim=60, input_length=max_word_count))
    model.add(GRU(units=256, return_sequences=True))
    model.add(MaxPooling1D())
    model.add(Flatten())
    model.add(Dense(250, activation='relu'))
    model.add(Dense(2, activation='softmax'))
#     model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    adam_optimizer = Adam(lr=0.001, decay=0.0001)
    model.compile(loss='categorical_crossentropy', optimizer=adam_optimizer, metrics=[metrics.CategoricalAccuracy(), metrics.AUC(), metrics.Precision(), metrics.Recall()])
    
#     print(model.summary())
    # ################## Deep Neural Network Model ###################### #

    best_accuracy = 0
    best_loss = 100000
    best_epoch = 0

    epoch_history = {
        'accuracy': [],
        'val_accuracy': [],
        'loss': [],
        'val_loss': [],
        'auc': [],
        'val_auc': []
    }

    history = model.fit(x=x_train_n_validation, y=y_train_n_validation, epochs=MAX_EPOCHS, batch_size=30, verbose=1, shuffle=True)
    scores = model.evaluate(x_test, y_test, verbose=0)
    print(model.metrics_names)
    cv_scores.append(scores)
    print("scores:", scores)
    fold += 1

In [ ]:
mean_accuracy = 0
mean_auc = 0
mean_precision = 0
mean_recall = 0
for score in cv_scores:
    mean_accuracy += score[1]
    mean_auc += score[2]
    mean_precision += score[3]
    mean_recall += score[4]
mean_accuracy = mean_accuracy / FOLDS_COUNT
mean_auc = mean_auc / FOLDS_COUNT
mean_precision = mean_precision / FOLDS_COUNT
mean_recall = mean_recall / FOLDS_COUNT
mean_f_score = 2*mean_precision*mean_recall/(mean_precision + mean_recall)
print("mean_accuracy", mean_accuracy)
print("mean_auc", mean_auc)
print("mean_precision", mean_precision)
print("mean_recall", mean_recall)
print("mean_f_score", mean_f_score)

# Ensemble Model

In [ ]:
def get_lstm_model():
    model = Sequential()
    model.add(Embedding(input_dim=dictionary_length, output_dim=60, input_length=max_word_count))
    model.add(LSTM(600))
    model.add(Dense(units=max_word_count, activation='tanh', kernel_regularizer=regularizers.l2(0.04),
                    activity_regularizer=regularizers.l2(0.015)))
    model.add(Dense(units=max_word_count, activation='relu', kernel_regularizer=regularizers.l2(0.01),
                    bias_regularizer=regularizers.l2(0.01)))
    model.add(Dense(2, activation='softmax', kernel_regularizer=regularizers.l2(0.001)))
    adam_optimizer = Adam(lr=0.001, decay=0.0001)
    model.compile(loss='binary_crossentropy', optimizer=adam_optimizer, metrics=[metrics.CategoricalAccuracy(), metrics.AUC(), metrics.Precision(), metrics.Recall()])

    return model

In [ ]:
def get_cnn_model():
    model = Sequential()
    model.add(Embedding(input_dim=dictionary_length, output_dim=60, input_length=max_word_count))
    model.add(Conv1D(32, 3, padding='same', activation='relu'))
    model.add(MaxPooling1D())
    model.add(Flatten())
    model.add(Dense(250, activation='relu'))
    model.add(Dense(2, activation='sigmoid'))
    adam_optimizer = Adam(lr=0.001, decay=0.0001)
    model.compile(loss='binary_crossentropy', optimizer=adam_optimizer, metrics=[metrics.CategoricalAccuracy(), metrics.AUC(), metrics.Precision(), metrics.Recall()])
    return model

In [ ]:
def get_bi_gru_model():
    model = Sequential()
    model.add(Embedding(input_dim=dictionary_length, output_dim=60, input_length=max_word_count))
    model.add(GRU(units=256, return_sequences=True))
    model.add(MaxPooling1D())
    model.add(Flatten())
    model.add(Dense(250, activation='relu'))
    model.add(Dense(2, activation='sigmoid'))
    adam_optimizer = Adam(lr=0.001, decay=0.0001)
    model.compile(loss='binary_crossentropy', optimizer=adam_optimizer, metrics=[metrics.CategoricalAccuracy(), metrics.AUC(), metrics.Precision(), metrics.Recall()])
    return model

In [ ]:
import numpy as np
from scipy import stats
def ensemble_pred(models, test_set):
    predictions = []
    for model in models:
        pred = model.predict(test_set, batch_size = 32)
        pred = list(np.argmax(pred, axis=1))
#         print(pred)
        predictions.append(pred)
    predictions_np = np.array(predictions)
    m = stats.mode(predictions_np)
    return m.mode[0].tolist()

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

def evaluate_perf(pred, actual):
    actual_list = [i[1] for i in actual]
    accuracy = accuracy_score(actual_list, pred)
    precision = precision_score(actual_list, pred, average='macro')
    recall = recall_score(actual_list, pred, average='macro')
    f_score = f1_score(actual_list, pred, average='macro')
    auc = roc_auc_score(actual_list, pred)
    return { "accuracy": accuracy, "precision": precision, "recall": recall, "f_score": f_score, 'auc': auc}

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import numpy
from sklearn.metrics import classification_report

cvscores = []
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
y_corpus_raw = [0 if cls[1] == 1 else 1 for cls in y_corpus]

for train_n_validation_indexes, test_indexes  in kfold.split(x_corpus, y_corpus_raw):
    x_train_n_validation = x_corpus[train_n_validation_indexes]
    y_train_n_validation = y_corpus[train_n_validation_indexes]
    x_test = x_corpus[test_indexes]
    y_test = y_corpus[test_indexes]

    # train and validation data sets
#     x_train, x_valid, y_train, y_valid = train_test_split(x_train_n_validation, y_train_n_validation,
#                                                           test_size=VALIDATION_TEST_SIZE, random_state=94)
    lstm_model = get_lstm_model()
    cnn_model = get_cnn_model()
    gru_model = get_bi_gru_model()
#     MAX_EPOCHS
    lstm_model.fit(x_train_n_validation, y_train_n_validation, epochs=MAX_EPOCHS, batch_size=10, verbose=1)
    cnn_model.fit(x_train_n_validation, y_train_n_validation, epochs=MAX_EPOCHS, batch_size=10, verbose=1)
    gru_model.fit(x_train_n_validation, y_train_n_validation, epochs=MAX_EPOCHS, batch_size=10, verbose=1)
#     model.fit(x_train, y_train, epochs=3, batch_size=10, verbose=1)
        # evaluate the model
#     scores = model.evaluate(x_test, y_test, verbose=0)
#     print(model.metrics_names)
#     print(scores)
#     print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    prediction = ensemble_pred([lstm_model, cnn_model, gru_model], x_test)
    print(classification_report(list(np.argmax(y_test, axis=1)), prediction, labels=[0, 1]))
    scores = evaluate_perf(prediction, y_test)
    print("score: ", scores)
    cvscores.append(scores)

In [ ]:
acc = []
pre = []
recall = []
fscore = []
auc = []
for i in cvscores:
    acc.append(i['accuracy'])
    pre.append(i['precision'])
    recall.append(i['recall'])
    fscore.append(i['f_score'])
    auc.append(i['auc'])

print("Accuracy:", sum(acc)/len(acc))
print("Precision:", sum(pre)/len(pre))
print("Recall:", sum(recall)/len(recall))
print("fScore:", sum(fscore)/len(fscore))
print("auc:", sum(auc)/len(auc))